# 🚀 Day 2-6: 비지도 학습의 시작, 군집화 (Clustering)

지금까지 우리는 '정답'이 있는 데이터를 가지고 모델을 학습시키는 <u>지도 학습(Supervised Learning)</u>, 즉 회귀와 분류에 대해 배웠습니다. 

모델은 `SalePrice`나 `Churn`과 같은 정답 레이블을 보고 패턴을 학습했죠.

하지만 세상의 많은 데이터에는 정답이 없습니다. 

예를 들어, 수많은 고객 데이터가 있을 때 이들을 비슷한 그룹으로 묶어 새로운 마케팅 전략을 세우고 싶다고 해봅시다. 

어떤 고객이 '우수 고객'이고 어떤 고객이 '이탈 위험 고객'인지 미리 정해진 답이 없습니다. 

이때 사용하는 것이 바로 <u>비지도 학습(Unsupervised Learning)</u> 입니다.

<u>군집화(Clustering)</u> 는 비지도 학습의 가장 대표적인 방법으로, 데이터의 숨겨진 구조나 패턴을 발견하여 비슷한 특성을 가진 데이터끼리 그룹으로 묶어주는 기법입니다. 

마치 이름표 없는 과일 상자에서 사과는 사과끼리, 바나나는 바나나끼리 모으는 것과 같습니다.

이번 시간에는 가장 널리 사용되는 두 가지 군집화 알고리즘, <u>K-Means</u>와 <u>계층적 군집(Hierarchical Clustering)</u> 에 대해 배웁니다. 

이 알고리즘들을 통해 고객 데이터를 분석하여 의미 있는 고객 세그먼트를 발굴하는 실습을 진행하겠습니다.

이번 실습에서는 Kaggle의 <u>고객 성향 분석(Customer Personality Analysis) 데이터셋</u>을 사용합니다. 

고객의 인구통계 정보, 소비 패턴 등을 담고 있어 군집화 실습에 매우 적합합니다.

---

### 1. 데이터 불러오기 및 준비

군집화는 특히 데이터의 스케일에 민감하므로, 본격적인 분석에 앞서 데이터를 불러오고 필요한 전처리를 수행하는 것이 매우 중요합니다.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage

# 데이터셋 로드
path = '../datasets/ml/crm/marketing_campaign.csv'
df = pd.read_csv(path, sep='\t')

print("데이터셋 크기:", df.shape)
df.info()

데이터셋 크기: (2240, 29)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases 

`Income` 컬럼에 일부 결측치가 있고, `Dt_Customer`는 날짜 형식, `Education`과 `Marital_Status`는 문자열(object) 타입입니다. 

군집화는 기본적으로 숫자형 데이터에 대해 거리를 계산하므로, 적절한 전처리가 필요합니다.

#### 💻 코드로 알아보기: 데이터 전처리

이번 실습에서는 분석의 편의를 위해 수치형 데이터만 사용하고, 결측치는 중앙값으로 채우겠습니다. 

또한, 군집화 알고리즘은 각 변수의 스케일(단위)에 큰 영향을 받으므로 `StandardScaler`를 사용하여 모든 변수를 표준화합니다.

In [2]:
# 수치형 컬럼만 선택
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
df_numeric = df[numeric_features]

# 전처리 파이프라인 생성: 결측치 처리(중앙값) -> 스케일링
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 데이터 전처리 적용
X_processed = preprocessor.fit_transform(df_numeric)
X_processed = pd.DataFrame(X_processed, columns=numeric_features)

print("전처리 후 데이터 크기:", X_processed.shape)
print("결측치 개수 확인:", X_processed.isnull().sum().sum())

전처리 후 데이터 크기: (2240, 26)
결측치 개수 확인: 0


이제 데이터가 준비되었으니, 본격적으로 K-Means 군집화를 시작해 보겠습니다.

---

### 2. K-평균 군집화 (K-Means Clustering)

K-Means는 가장 널리 알려진 군집화 알고리즘 중 하나로, 구현이 간단하고 속도가 빨라 많이 사용됩니다. 이름의 'K'는 우리가 <u>비지도 학습(Unsupervised Learning)</u> 를 의미합니다.

#### 🧠 개념 이해하기

K-Means 알고리즘은 다음과 같은 순서로 동작합니다.

1.  <u>군집화(Clustering)</u>: 지정된 K개의 군집 중심점을 데이터 공간에 임의로 배치합니다.
2.  <u>K-Means</u>: 각 데이터 포인트를 가장 가까운 군집 중심점에 할당합니다. 이로써 K개의 군집이 형성됩니다.
3.  <u>계층적 군집(Hierarchical Clustering)</u>: 각 군집에 속한 데이터 포인트들의 평균을 계산하여 군집 중심점을 새로 업데이트합니다.
4.  <u>고객 성향 분석(Customer Personality Analysis) 데이터셋</u>: 군집 중심점이 더 이상 변하지 않을 때까지 2번과 3번 과정을 반복합니다.

    <div style="background: white; width: 800px;">
    <img src="https://blog.kakaocdn.net/dn/cuSJQH/btrAdYtGm2f/dCfKAKiurKkW3t5WDCkkKk/img.png">
    </div>

K-Means는 각 군집 내 데이터 포인트들과 군집 중심 간의 거리 제곱의 합, 즉 <u>미리 지정해야 하는 군집의 개수</u> 를 최소화하는 방향으로 학습합니다. 이너셔 값이 작을수록 군집 내 데이터들이 잘 뭉쳐있음을 의미합니다.

$$ \text{Inertia} = \sum_{i=1}^{k} \sum_{x \in C_i} ||x - \mu_i||^2 $$
($k$: 군집 수, $C_i$: $i$번째 군집, $\mu_i$: $i$번째 군집의 중심점)


- K-means++ 초기화 기법

    <u>군집 중심(Centroid) 초기화</u> 에서는 `k-means++` 라는 똑똑한 초기화 방식을 기본으로 사용하여, 임의 초기화로 인한 성능 저하 문제를 완화합니다.
    
    <u>할당(Assignment)</u>는 K-means 군집화 알고리즘에서 중심점(centroid) 초기화의 단점을 보완하기 위해 제안된 방법입니다. 

    기존 K-means의 무작위 초기화는 결과의 품질과 일관성에 큰 영향을 미치며, 지역 최적해에 빠질 가능성이 높았습니다. 

    K-means++는 이러한 문제를 해결해 더 안정적이고 품질 좋은 군집화를 가능하게 합니다.

    알고리즘

    1. 첫 번째 중심점 선택: 데이터 포인트 중 하나를 무작위로 선택해 첫 번째 중심점으로 지정합니다.
    2. 다음 중심점 선택: 나머지 데이터 포인트 각각에 대해, 이미 선택된 중심점들과의 거리 중 가장 가까운 거리의 제곱을 계산합니다. 이 값을 확률 분포로 사용해, 거리가 멀수록(즉, 기존 중심점과 멀리 떨어진 데이터일수록) 다음 중심점으로 선택될 확률이 높아집니다.
    3. 반복: 위 과정을 K개의 중심점이 모두 선택될 때까지 반복합니다.
    4. K-means 알고리즘 실행: 이렇게 선택된 K개의 중심점을 바탕으로 일반적인 K-means 알고리즘(클러스터 할당 및 중심점 업데이트)을 수행합니다.

In [3]:
# K-Means 모델 생성 및 학습 (예시로 K=4로 설정)
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
kmeans.fit(X_processed)

# 각 데이터에 할당된 군집 레이블 확인
cluster_labels = kmeans.labels_
print("군집 레이블 (처음 10개):", cluster_labels[:10])

# 최종 이너셔 값 확인
print("\n최종 이너셔 값:", kmeans.inertia_)

군집 레이블 (처음 10개): [1 0 1 0 0 3 3 0 0 0]

최종 이너셔 값: 36258.35171562955


#### 💻 코드로 알아보기: 군집 결과 시각화
군집화 결과는 시각화를 통해 직관적으로 이해하는 것이 좋습니다. 

하지만 우리 데이터는 다차원이므로 바로 시각화할 수 없습니다. 

<u>비지도 학습(Unsupervised Learning)</u>을 사용하여 데이터를 2차원으로 축소한 뒤, 각 군집을 다른 색으로 표현해 보겠습니다.

In [6]:
# PCA를 사용하여 2차원으로 차원 축소
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_processed)

# 시각화를 위한 데이터프레임 생성
df_pca = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_pca['Cluster'] = cluster_labels.astype(str) # 색상 구분을 위해 문자열로 변환

In [7]:

# 군집 중심점도 동일하게 PCA 변환
centroids_pca = pca.transform(kmeans.cluster_centers_)

# Plotly Express로 시각화
fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='Cluster', 
                 title='K-Means Clustering Result (K=4)',
                 labels={'Cluster': '고객 군집'})

# 군집 중심점 추가 (검은색 X 마커)
fig.add_trace(go.Scatter(x=centroids_pca[:, 0], y=centroids_pca[:, 1],
                           mode='markers', marker_symbol='x', 
                           marker_color='black', marker_size=12, 
                           name='Centroids'))

fig.show()

c:\Users\Admin\workspace\analysis_study\class\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names



#### 🧠 개념 이해하기: 최적의 K 찾기 (Elbow Method & Silhouette Score)

K-Means의 가장 큰 숙제는 '적절한 K를 어떻게 찾을 것인가?' 입니다.

K가 너무 작으면 다양한 특성의 데이터가 하나의 군집에 묶이고, 너무 크면 하나의 군집이 불필요하게 여러 개로 쪼개집니다.
 
이를 해결하기 위한 대표적인 두 가지 방법이 있습니다.

1.  <u>비지도 학습(Unsupervised Learning)</u>
   
    K를 1부터 점차 늘려가면서 이너셔 값의 변화를 관찰합니다. 
    
    K가 커질수록 이너셔는 당연히 줄어들지만, 어느 순간부터 감소폭이 급격히 완만해지는 지점이 나타납니다. 
    
    이 지점이 마치 '팔꿈치(Elbow)'처럼 꺾여 보이며, 이 지점의 K를 최적의 값으로 선택하는 방법입니다.

2.  <u>군집화(Clustering)</u>
    
    군집이 얼마나 잘 형성되었는지를 평가하는 지표입니다. 
    
    각 데이터 포인트에 대해 '자신이 속한 군집 내 데이터들과 얼마나 가까운지(응집도)'와 '가장 가까운 다른 군집의 데이터들과 얼마나 먼지(분리도)'를 함께 고려합니다. 
    
    점수는 -1에서 1 사이의 값을 가지며, 1에 가까울수록 군집이 잘 분리되었음을 의미합니다.

In [8]:
# Elbow Method를 위한 이너셔 값 계산
inertia_list = []
k_range = range(2, 11) # K를 2부터 10까지 변화

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X_processed)
    inertia_list.append(kmeans.inertia_)

In [9]:
# Elbow Method 시각화
fig = px.line(x=k_range, y=inertia_list, title='Elbow Method for Optimal K',
              markers=True, labels={'x': 'Number of Clusters (K)', 'y': 'Inertia'})
fig.show()

In [11]:
# Silhouette Score 계산 : elobow method 아이디어는 비슷 / 두 가지 값을 보고 적절한 k 찾기
silhouette_list = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    cluster_labels = kmeans.fit_predict(X_processed)
    score = silhouette_score(X_processed, cluster_labels)
    silhouette_list.append(score)

# Silhouette Score 시각화
fig = px.line(x=k_range, y=silhouette_list, title='Silhouette Score for Optimal K',
              markers=True, labels={'x': 'Number of Clusters (K)', 'y': 'Silhouette Score'})
fig.show()

엘보우 그래프에서는 K=4 또는 K=5 근처에서 기울기가 완만해지는 것을 볼 수 있습니다. 

실루엣 점수는 K=3 또는 K=4에서 높은 값을 보입니다. 종합적으로 <u>K=4</u>가 합리적인 선택으로 보입니다.


#### ✏️ 연습문제 1
K-Means 모델을 `n_clusters=5`로 설정하여 학습시킨 후, 최종 이너셔(inertia) 값을 출력하고, PCA를 이용해 군집화 결과를 시각화해보세요.

In [12]:
# 연습문제 1 코드
kmeans_5 = KMeans(n_clusters=5, random_state=42, n_init='auto')
cluster_labels_5 = kmeans_5.fit_predict(X_processed)
print(f"K=5일 때의 이너셔 값: {kmeans_5.inertia_:.2f}")

# 시각화를 위한 데이터프레임 생성
df_pca_5 = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_pca_5['Cluster'] = cluster_labels_5.astype(str)

# 군집 중심점도 동일하게 PCA 변환
centroids_pca_5 = pca.transform(kmeans_5.cluster_centers_)

fig = px.scatter(df_pca_5, x='PCA1', y='PCA2', color='Cluster', title='K-Means Clustering Result (K=5)')
fig.add_trace(go.Scatter(x=centroids_pca_5[:, 0], y=centroids_pca_5[:, 1], mode='markers', marker_symbol='x', marker_color='black', marker_size=12, name='Centroids'))
fig.show()

K=5일 때의 이너셔 값: 34846.76


c:\Users\Admin\workspace\analysis_study\class\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names



#### ✏️ 연습문제 2
`K=4`로 군집화를 수행한 결과를 바탕으로, 각 군집의 특성을 파악해보세요. 전처리 전의 원본 `df` 데이터프레임에 군집 레이블을 추가한 뒤, `groupby()`를 사용하여 각 군집별 `Income`, `MntWines`(와인 구매액), `NumWebPurchases`(웹 구매 횟수)의 평균을 계산하고 어떤 차이가 있는지 설명해보세요.

In [13]:
# 연습문제 2 코드
# 힌트: df['Cluster'] = cluster_labels
# df.groupby('Cluster')[['Income', 'MntWines', 'NumWebPurchases']].mean()
# K=4로 다시 학습
kmeans_4 = KMeans(n_clusters=4, random_state=42, n_init='auto')
cluster_labels_4 = kmeans_4.fit_predict(X_processed)

# 원본 데이터프레임에 군집 레이블 추가
df_analysis = df.copy()
df_analysis['Cluster'] = cluster_labels_4

# 군집별 특성 평균 계산
cluster_profile = df_analysis.groupby('Cluster')[['Income', 'MntWines', 'NumWebPurchases']].mean().round(2)
print(cluster_profile)

print("\n--- 분석 결과 ---")
print("Cluster 0: 소득이 가장 낮고 와인 소비량도 적은 '저소득 고객' 그룹입니다.")
print("Cluster 1: 소득이 높고 와인 소비량이 많은 '프리미엄 고객' 그룹입니다.")
print("Cluster 2: 소득이 가장 높고 와인 소비량이 압도적으로 많은 'VIP 고객' 그룹입니다.")
print("Cluster 3: 소득은 중간 수준이지만 웹 구매가 가장 활발한 '온라인 쇼핑족' 그룹입니다.")

           Income  MntWines  NumWebPurchases
Cluster                                     
0        34643.06     40.31             2.10
1        74192.63    490.99             5.20
2        81926.74    874.70             5.44
3        57477.26    454.22             6.29

--- 분석 결과 ---
Cluster 0: 소득이 가장 낮고 와인 소비량도 적은 '저소득 고객' 그룹입니다.
Cluster 1: 소득이 높고 와인 소비량이 많은 '프리미엄 고객' 그룹입니다.
Cluster 2: 소득이 가장 높고 와인 소비량이 압도적으로 많은 'VIP 고객' 그룹입니다.
Cluster 3: 소득은 중간 수준이지만 웹 구매가 가장 활발한 '온라인 쇼핑족' 그룹입니다.



### 3. 계층적 군집 (Hierarchical Clustering)

K-Means와 달리, 계층적 군집은 군집의 수 K를 미리 정할 필요가 없습니다.

대신 데이터 간의 거리를 기반으로 나무 형태의 계층 구조를 만들어내는 <u>비지도 학습(Unsupervised Learning)</u> 기법입니다.

---

#### 🧠 개념 이해하기

계층적 군집에는 두 가지 방식이 있습니다:

1. <u>병합형(Agglomerative)</u>:

   * 가장 일반적으로 사용되는 방식입니다.
   * 모든 데이터를 각각 하나의 군집으로 시작하여, 가장 가까운 두 군집을 반복적으로 병합합니다.
   * 군집 수가 1이 될 때까지 병합을 계속합니다.

2. <u>분할형(Divisive)</u>:

   * 하나의 전체 군집에서 시작하여, 점차 데이터를 나누며 군집을 형성하는 방식입니다.
   * 상대적으로 잘 사용되지 않습니다.

---

#### 🧪 군집 간 거리 측정 방식

군집 간의 유사도(혹은 거리)를 측정하는 방법은 여러 가지가 있으며, 이 방식에 따라 최종 군집 결과가 달라집니다:

| 방법                           | 설명                       |
| ---------------------------- | ------------------------ |
| <u>단일 연결 (Single Linkage)</u>   | 두 군집에서 가장 가까운 두 점 사이의 거리 |
| <u>완전 연결 (Complete Linkage)</u> | 두 군집에서 가장 먼 두 점 사이의 거리   |
| <u>평균 연결 (Average Linkage)</u>  | 두 군집 내 모든 점쌍의 평균 거리      |
| <u>중심 연결 (Centroid Linkage)</u> | 각 군집의 중심점(centroid) 간 거리 |

---

#### 📊 시각화: 덴드로그램 (Dendrogram)

계층적 군집의 결과를 시각화할 때는 <u>덴드로그램(Dendrogram)</u> 을 사용합니다.
덴드로그램은 나무 구조로 이루어져 있으며 다음과 같은 정보를 담고 있습니다:

* <u>X축</u>: 데이터 포인트 또는 군집
* <u>Y축</u>: 군집 간의 거리 또는 유사도

덴드로그램에서 긴 수직선은 군집 간 거리가 멀다는 뜻입니다.
이러한 선을 '가위로 자르듯이' 적절한 높이에서 끊어주면, 그 높이 기준으로 군집 수를 결정할 수 있습니다.

---

#### 📁 예시 데이터: 고객 성향 분석 데이터셋 (Customer Personality Analysis)

예를 들어, 마케팅 데이터를 활용해 고객을 성향에 따라 나누고 싶을 때 계층적 군집을 사용할 수 있습니다.
K 값을 지정할 필요 없이, 덴드로그램을 통해 군집 수를 유연하게 탐색할 수 있다는 점이 큰 장점입니다.

---

#### ✅ 요약

* 계층적 군집은 <u>K 값을 미리 정하지 않아도 되는</u> 유연한 군집화 기법입니다.
* 병합형 방식이 일반적으로 사용되며, 다양한 거리 계산 방법이 존재합니다.
* <u>덴드로그램을 통해 시각적으로 군집 수를 결정</u>할 수 있습니다.
* 대규모 데이터에는 계산 복잡도가 크기 때문에 <u>소규모 또는 중간 규모 데이터셋에 적합</u>합니다.

#### 💻 코드로 알아보기: 덴드로그램 생성 및 군집화
덴드로그램은 `plotly.figure_factory`의 `create_dendrogram` 함수를 사용하여 생성할 수 있습니다. 

데이터가 많으면 덴드로그램이 복잡해지므로, 일부 데이터(30개)만 샘플링하여 시각화해 보겠습니다.

In [14]:
X_processed

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,-0.020999,-0.985345,0.235696,-0.825218,-0.929894,0.307039,0.983781,1.551577,1.679702,2.462147,...,0.693904,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,2.388846
1,-1.053058,-1.235733,-0.235454,1.032559,0.906934,-0.383664,-0.870479,-0.636301,-0.713225,-0.650449,...,-0.130463,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
2,-0.447070,-0.317643,0.773999,-0.825218,-0.929894,-0.798086,0.362723,0.570804,-0.177032,1.345274,...,-0.542647,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
3,0.181716,1.268149,-1.022355,1.032559,-0.929894,-0.798086,-0.870479,-0.560857,-0.651187,-0.503974,...,0.281720,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
4,-0.082614,1.017761,0.241888,1.032559,-0.929894,1.550305,-0.389085,0.419916,-0.216914,0.155164,...,-0.130463,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1.625983,-0.150717,0.358936,-0.825218,0.906934,-0.107383,1.203678,0.419916,0.066692,0.081926,...,-0.130463,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
2236,-0.490200,-1.903435,0.470432,2.890335,0.906934,0.237969,0.303291,-0.661449,-0.606873,-0.687068,...,0.693904,-0.28014,-0.283830,-0.28014,3.815174,-0.11651,-0.097282,0.0,0.0,-0.418612
2237,0.516905,1.017761,0.189476,-0.825218,-0.929894,1.446700,1.795020,0.545656,0.221789,-0.101168,...,0.281720,-0.28014,3.523233,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
2238,0.814199,-1.068807,0.679401,-0.825218,0.906934,-1.419719,0.368666,0.092992,0.208495,0.777683,...,-0.954831,-0.28014,-0.283830,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612


In [11]:
from plotly.figure_factory import create_dendrogram
# Ward 연결법을 사용한 덴드로그램 생성
# 데이터가 많으므로 일부만 샘플링하여 시각화
X_sample = X_processed.sample(n=30, random_state=42)

fig = create_dendrogram(X_sample, linkagefun=lambda x: linkage(x, method='ward'))
fig.update_layout(title_text='Dendrogram with Ward Linkage (Sampled Data)',
                  yaxis_title='Distance')
fig.show()

덴드로그램을 보고 적절한 군집 수를 결정한 뒤 (e.g., y축 거리가 긴 부분을 잘라 3개 군집으로 결정), `scikit-learn`의 `AgglomerativeClustering`으로 군집화를 수행할 수 있습니다.

In [14]:
# 계층적 군집 모델 생성 및 학습 (K=4로 설정)
agg_cluster = AgglomerativeClustering(n_clusters=3, linkage='ward')
agg_labels = agg_cluster.fit_predict(X_processed)

# PCA를 이용한 시각화
df_pca['Agg_Cluster'] = agg_labels.astype(str)

fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='Agg_Cluster',
                 title='Hierarchical Clustering Result (K=4, Ward)',
                 labels={'Agg_Cluster': '고객 군집'})
fig.show()

#### ✏️ 연습문제 3
`AgglomerativeClustering` 모델의 `linkage` 파라미터를 `'average'`로 변경하여 군집화를 수행하고, PCA를 이용해 시각화한 뒤 'ward' 연결법의 결과와 어떻게 다른지 비교해보세요.

In [15]:
# 연습문제 3 코드
# 'average' 연결법으로 계층적 군집 수행
agg_cluster_avg = AgglomerativeClustering(n_clusters=4, linkage='average')
agg_labels_avg = agg_cluster_avg.fit_predict(X_processed)

# 시각화
df_pca_agg_avg = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_pca_agg_avg['Agg_Cluster_Avg'] = agg_labels_avg.astype(str)

fig = px.scatter(df_pca_agg_avg, x='PCA1', y='PCA2', color='Agg_Cluster_Avg',
                 title='Hierarchical Clustering Result (K=4, Average)',
                 labels={'Agg_Cluster_Avg': '고객 군집'})
fig.show()

print("--- 비교 분석 ---")
print("'ward' 연결법은 군집의 크기를 비교적 균등하게 나누려는 경향이 있는 반면, 'average' 연결법은 하나의 거대한 군집과 여러 개의 작은 군집들로 나뉘는 경향을 보입니다. 이는 'average'가 이상치에 덜 민감하기 때문일 수 있습니다. 데이터의 특성에 따라 적절한 연결법을 선택하는 것이 중요합니다.")

--- 비교 분석 ---
'ward' 연결법은 군집의 크기를 비교적 균등하게 나누려는 경향이 있는 반면, 'average' 연결법은 하나의 거대한 군집과 여러 개의 작은 군집들로 나뉘는 경향을 보입니다. 이는 'average'가 이상치에 덜 민감하기 때문일 수 있습니다. 데이터의 특성에 따라 적절한 연결법을 선택하는 것이 중요합니다.


#### ✏️ 연습문제 4
위에서 생성한 <u>비지도 학습(Unsupervised Learning)</u>을 시각적으로 분석하여, 군집을 3개로 나누는 것이 적절한지 혹은 5개로 나누는 것이 적절한지 여러분의 생각을 설명해보세요. (정답은 없습니다. 덴드로그램을 해석하는 능력을 기르는 문제입니다.)

<u>군집화(Clustering)</u>: 덴드로그램에서 수직선의 길이가 길수록 두 클러스터가 합쳐질 때의 거리가 멀다는 뜻입니다. 즉, 오랜 시간 동안 다른 클러스터와 합쳐지지 않고 독립적으로 유지되었다는 의미로 해석할 수 있습니다. 일반적으로 수직선이 가장 긴 부분을 자르는 것이 좋은 기준으로 여겨집니다.

In [16]:
print("--- 덴드로그램 해석 (예시 답안) ---")
print("덴드로그램을 보면, y축 값이 약 25~30 사이에서 가장 긴 수직선이 나타납니다. 이 선을 자르면 크게 2개의 군집으로 나뉩니다. 그 다음으로 긴 수직선은 약 15~20 사이에서 나타나며, 이 선들을 자르면 3개 또는 4개의 군집으로 나눌 수 있습니다.")
print("만약 군집을 3개로 나눈다면 (y=20 근처를 자름), 매우 뚜렷하게 구분되는 세 그룹을 얻을 수 있습니다. 군집을 5개로 나누려면 (y=10~12 근처를 자름), 기존의 큰 군집 하나를 더 세분화해야 하는데, 이때 합쳐지는 군집 간의 거리가 상대적으로 짧아 군집의 특성이 뚜렷하지 않을 수 있습니다.")
print("따라서, 이 덴드로그램만 본다면 3개 또는 4개의 군집이 비교적 합리적인 선택으로 보이며, 5개는 다소 과적합(지나치게 세분화)의 위험이 있어 보입니다.")

--- 덴드로그램 해석 (예시 답안) ---
덴드로그램을 보면, y축 값이 약 25~30 사이에서 가장 긴 수직선이 나타납니다. 이 선을 자르면 크게 2개의 군집으로 나뉩니다. 그 다음으로 긴 수직선은 약 15~20 사이에서 나타나며, 이 선들을 자르면 3개 또는 4개의 군집으로 나눌 수 있습니다.
만약 군집을 3개로 나눈다면 (y=20 근처를 자름), 매우 뚜렷하게 구분되는 세 그룹을 얻을 수 있습니다. 군집을 5개로 나누려면 (y=10~12 근처를 자름), 기존의 큰 군집 하나를 더 세분화해야 하는데, 이때 합쳐지는 군집 간의 거리가 상대적으로 짧아 군집의 특성이 뚜렷하지 않을 수 있습니다.
따라서, 이 덴드로그램만 본다면 3개 또는 4개의 군집이 비교적 합리적인 선택으로 보이며, 5개는 다소 과적합(지나치게 세분화)의 위험이 있어 보입니다.


---

### 4. DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

K-Means가 구형(spherical)의 군집을 가정하는 반면, <u>DBSCAN</u>은 데이터의 <u>밀도(density)</u>를 기반으로 군집을 형성합니다. 

K-Means와 달리 군집의 개수 K를 미리 지정할 필요가 없으며, 기하학적으로 복잡한 모양의 군집도 잘 찾아내고, 

어떤 군집에도 속하지 않는 <u>노이즈(Noise) 또는 이상치(Outlier)를 구분</u>해내는 강력한 장점이 있습니다.

#### 🧠 개념 이해하기

DBSCAN을 이해하기 위해서는 세 가지 포인트 유형과 두 개의 핵심 파라미터를 알아야 합니다.

<u>핵심 파라미터:</u>
* `eps` (epsilon, 입실론): 한 데이터 포인트로부터의 거리를 정의하는 반경입니다. 이 반경 안에 들어오는 다른 데이터 포인트들은 '이웃'으로 간주됩니다.
  
* `min_samples`: 하나의 포인트가 '핵심 포인트'가 되기 위해 `eps` 반경 내에 포함해야 하는 최소한의 이웃 데이터 개수 (자기 자신 포함) 입니다.

<u>포인트 유형:</u>
1.  <u>핵심 포인트 (Core Point)</u>: `eps` 반경 내에 `min_samples` 개 이상의 이웃을 가진 포인트입니다. 군집의 '중심부'를 형성합니다.
   
2.  <u>경계 포인트 (Border Point)</u>: 핵심 포인트는 아니지만, `eps` 반경 내에 핵심 포인트가 있는 포인트입니다. 군집의 '가장자리'에 위치합니다.
3.  <u>노이즈 (Noise Point)</u>: 핵심 포인트도, 경계 포인트도 아닌 포인트입니다. 밀도가 낮은 지역에 고립되어 있는 이상치로 간주됩니다.

DBSCAN은 임의의 점에서 시작하여, 그 점이 핵심 포인트이면 `eps` 반경 내의 모든 점들을 하나의 군집으로 묶는 과정을 재귀적으로 반복하여 군집을 확장해 나갑니다.

![DBSCAN 알고리즘 예시](https://pic2.zhimg.com/v2-58145667049e230f95e07c3dfbfd31ad_b.gif)


#### 💻 코드로 알아보기: DBSCAN 적용

`eps`와 `min_samples` 값에 따라 군집 결과가 크게 달라지므로, 적절한 값을 찾는 것이 중요합니다. 보통 `min_samples`는 데이터의 차원을 고려하여 먼저 정한 뒤, `eps` 값을 조정하며 최적의 결과를 찾습니다.

In [18]:
from sklearn.cluster import DBSCAN

# DBSCAN 모델 생성 및 학습
# eps=2.5, min_samples=10으로 설정 (여러 번의 실험을 통해 찾은 값)
dbscan = DBSCAN(eps=2.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_processed)

# 노이즈는 -1로 레이블링됩니다.
n_clusters_ = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise_ = list(dbscan_labels).count(-1)

print(f"DBSCAN이 찾은 군집 개수: {n_clusters_}")
print(f"노이즈(이상치)로 판단된 데이터 개수: {n_noise_}")

# PCA를 이용한 시각화
df_pca['DBSCAN_Cluster'] = dbscan_labels.astype(str)

fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='DBSCAN_Cluster',
                 title='DBSCAN Clustering Result (eps=2.5, min_samples=10)',
                 labels={'DBSCAN_Cluster': '고객 군집'})
fig.show()

DBSCAN이 찾은 군집 개수: 6
노이즈(이상치)로 판단된 데이터 개수: 871


In [19]:
# PCA 3차원 시각화
from sklearn.decomposition import PCA

# 3차원 PCA 수행
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_processed)

# 3차원 PCA 결과를 데이터프레임으로 변환
df_pca_3d = pd.DataFrame(X_pca_3d, columns=['PCA1', 'PCA2', 'PCA3'])
df_pca_3d['Cluster'] = dbscan_labels.astype(str)

# 3차원 PCA 시각화
fig = px.scatter_3d(df_pca_3d, x='PCA1', y='PCA2', z='PCA3', 
                    color='Cluster',
                    title='DBSCAN Clustering - 3D PCA Visualization',
                    labels={'Cluster': '고객 군집'})
fig.update_layout(scene=dict(
    xaxis_title='첫 번째 주성분',
    yaxis_title='두 번째 주성분', 
    zaxis_title='세 번째 주성분'
))
fig.show()

# t-SNE 3차원 시각화
from sklearn.manifold import TSNE

# 3차원 t-SNE 수행
tsne_3d = TSNE(n_components=3, random_state=42, perplexity=30)
X_tsne_3d = tsne_3d.fit_transform(X_processed)

# 3차원 t-SNE 결과를 데이터프레임으로 변환
df_tsne_3d = pd.DataFrame(X_tsne_3d, columns=['tSNE1', 'tSNE2', 'tSNE3'])
df_tsne_3d['Cluster'] = dbscan_labels.astype(str)

# 3차원 t-SNE 시각화
fig = px.scatter_3d(df_tsne_3d, x='tSNE1', y='tSNE2', z='tSNE3',
                    color='Cluster',
                    title='DBSCAN Clustering - 3D t-SNE Visualization',
                    labels={'Cluster': '고객 군집'})
fig.update_layout(scene=dict(
    xaxis_title='첫 번째 t-SNE 차원',
    yaxis_title='두 번째 t-SNE 차원',
    zaxis_title='세 번째 t-SNE 차원'
))
fig.show()

# 설명력 비교
print("PCA 설명력:")
print(f"첫 번째 주성분: {pca_3d.explained_variance_ratio_[0]:.3f}")
print(f"두 번째 주성분: {pca_3d.explained_variance_ratio_[1]:.3f}")
print(f"세 번째 주성분: {pca_3d.explained_variance_ratio_[2]:.3f}")
print(f"총 설명력: {sum(pca_3d.explained_variance_ratio_):.3f}")

PCA 설명력:
첫 번째 주성분: 0.271
두 번째 주성분: 0.084
세 번째 주성분: 0.079
총 설명력: 0.435


#### ✏️ 연습문제 5

DBSCAN의 파라미터를 `eps=3.0`, `min_samples=15`로 변경하여 다시 군집화를 수행해보세요. 찾은 군집의 개수와 노이즈 데이터의 개수가 어떻게 변하는지 관찰하고, 왜 그런 결과가 나왔는지 설명해보세요.

<u>DBSCAN</u>: `eps`가 커지면 하나의 군집이 더 넓은 영역을 포괄하게 되고, `min_samples`가 커지면 핵심 포인트가 되기 위한 조건이 더 까다로워집니다.

In [20]:
# 연습문제 5 코드
dbscan_new = DBSCAN(eps=3.0, min_samples=15)
dbscan_labels_new = dbscan_new.fit_predict(X_processed)

n_clusters_new = len(set(dbscan_labels_new)) - (1 if -1 in dbscan_labels_new else 0)
n_noise_new = list(dbscan_labels_new).count(-1)

print(f"새로운 파라미터로 찾은 군집 개수: {n_clusters_new}")
print(f"새로운 파라미터로 판단된 노이즈 개수: {n_noise_new}")

print("\n--- 분석 결과 ---")
print("eps가 2.5에서 3.0으로 증가하면서, 각 포인트가 더 넓은 반경을 이웃으로 간주하게 됩니다. \n이로 인해 기존에 분리되었던 군집들이 하나로 합쳐져 전체 군집의 개수가 줄어들었습니다.\n 또한, 더 많은 포인트들이 군집에 속하게 되어 노이즈의 개수도 감소했습니다.\n 반면 min_samples가 10에서 15로 증가하여 핵심 포인트가 되기 더 어려워졌지만, eps 증가 효과가 더 커서 전체적으로 더 큰 군집이 형성되었습니다.")

새로운 파라미터로 찾은 군집 개수: 3
새로운 파라미터로 판단된 노이즈 개수: 643

--- 분석 결과 ---
eps가 2.5에서 3.0으로 증가하면서, 각 포인트가 더 넓은 반경을 이웃으로 간주하게 됩니다. 
이로 인해 기존에 분리되었던 군집들이 하나로 합쳐져 전체 군집의 개수가 줄어들었습니다.
 또한, 더 많은 포인트들이 군집에 속하게 되어 노이즈의 개수도 감소했습니다.
 반면 min_samples가 10에서 15로 증가하여 핵심 포인트가 되기 더 어려워졌지만, eps 증가 효과가 더 커서 전체적으로 더 큰 군집이 형성되었습니다.


---

### 5. 가우시안 혼합 모델 (Gaussian Mixture Model, GMM)

<u>가우시안 혼합 모델</u>은 데이터가 여러 개의 <u>밀도(density)</u>가 섞여서 생성되었다고 가정하는 확률 기반의 군집화 알고리즘입니다.

#### 🧠 개념 이해하기

K-Means가 각 데이터를 하나의 군집에만 할당하는 <u>노이즈(Noise) 또는 이상치(Outlier)를 구분</u> 방식이라면, GMM은 각 데이터가 특정 군집에 속할 <u>핵심 파라미터:</u>을 계산하는 <u>포인트 유형:</u> 방식을 사용합니다. 예를 들어, 어떤 데이터가 'A 군집에 속할 확률 80%, B 군집에 속할 확률 20%'와 같이 표현됩니다.

이러한 특성 덕분에 GMM은 단순히 데이터를 나누는 것을 넘어, 데이터의 생성 확률 분포 자체를 모델링할 수 있습니다. K-Means처럼 원형 군집만 가정하지 않고, `covariance_type` 파라미터를 통해 타원형 등 더 유연한 모양의 군집을 찾아낼 수 있습니다.

In [21]:
from sklearn.mixture import GaussianMixture

# GMM 모델 생성 및 학습 (K-Means와 비교를 위해 K=4로 설정)
gmm = GaussianMixture(n_components=4, random_state=42)
gmm_labels = gmm.fit_predict(X_processed)

# 각 데이터가 각 군집에 속할 확률 확인
gmm_probs = gmm.predict_proba(X_processed)
print("각 군집에 속할 확률 (처음 5개 데이터):")
print(pd.DataFrame(gmm_probs.round(3), columns=[f'Cluster {i}' for i in range(4)]).head())

# PCA를 이용한 시각화
df_pca['GMM_Cluster'] = gmm_labels.astype(str)
fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='GMM_Cluster',
                 title='GMM Clustering Result (K=4)',
                 labels={'GMM_Cluster': '고객 군집'})
fig.show()

각 군집에 속할 확률 (처음 5개 데이터):
   Cluster 0  Cluster 1  Cluster 2  Cluster 3
0        0.0        0.0        0.0        1.0
1        1.0        0.0        0.0        0.0
2        0.0        0.0        0.0        1.0
3        1.0        0.0        0.0        0.0
4        0.0        0.0        0.0        1.0


#### ✏️ 연습문제 6

GMM 모델의 `covariance_type`을 `'spherical'`로 변경하여 학습하고, PCA로 시각화한 결과를 기본값('full')과 비교해보세요. 군집의 모양이 어떻게 변했나요?

<u>DBSCAN</u>: `covariance_type`은 군집의 모양을 결정합니다. 'spherical'은 군집이 원형이라고 가정하며, 'diag'는 축에 평행한 타원, 'full'은 방향에 제약이 없는 자유로운 타원을 가정합니다.

In [26]:
# 연습문제 6 코드
gmm_spherical = GaussianMixture(n_components=4, covariance_type='spherical', random_state=42)
gmm_labels_spherical = gmm_spherical.fit_predict(X_processed)

# PCA를 이용한 시각화
df_pca['GMM_Cluster_Spherical'] = gmm_labels_spherical.astype(str)
fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='GMM_Cluster_Spherical',
                 title='GMM Clustering Result (K=4, Covariance=Spherical)',
                 labels={'GMM_Cluster_Spherical': '고객 군집'})
fig.show()

---

### 6. BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies)

<u>BIRCH</u>는 대용량 데이터셋을 위해 특별히 설계된, 메모리 효율적인 군집화 알고리즘입니다. 모든 데이터를 메모리에 저장하는 대신, 데이터를 요약한 정보를 트리 구조로 저장하여 처리 속도를 획기적으로 개선했습니다.

#### 🧠 개념 이해하기

BIRCH의 핵심은 <u>밀도(density)</u> 입니다. 

1.  <u>노이즈(Noise) 또는 이상치(Outlier)를 구분</u>: BIRCH는 데이터를 스캔하면서 작은 마이크로 클러스터(CF)들의 통계적 요약 정보(점의 개수, 선형 합, 제곱 합)를 CF 트리에 저장합니다.
2.  <u>핵심 파라미터:</u>: 이 CF들은 계층적인 트리 구조를 형성합니다. 이 덕분에 모든 데이터를 메모리에 올리지 않고도 군집의 특성을 파악할 수 있습니다. (메모리 사용량 감소)
3.  <u>포인트 유형:</u>: CF 트리가 구축된 후, 트리의 리프 노드(가장 마지막의 요약 정보들)에 대해 일반적인 군집 알고리즘(e.g., 계층적 군집)을 적용하여 최종 군집을 형성합니다.

이러한 접근 방식 때문에 BIRCH는 한 번의 데이터 스캔만으로도 군집을 형성할 수 있어 <u>핵심 포인트 (Core Point)</u>합니다.

In [22]:
from sklearn.cluster import Birch

# BIRCH 모델 생성 및 학습 (K=4로 설정)
birch = Birch(n_clusters=4)
birch_labels = birch.fit_predict(X_processed)

print("BIRCH 군집 레이블 (처음 10개):", birch_labels[:10])

# PCA를 이용한 시각화
df_pca['BIRCH_Cluster'] = birch_labels.astype(str)
fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='BIRCH_Cluster',
                 title='BIRCH Clustering Result (K=4)',
                 labels={'BIRCH_Cluster': '고객 군집'})
fig.show()

BIRCH 군집 레이블 (처음 10개): [2 0 2 0 0 2 2 0 0 0]


#### ✏️ 연습문제 7

BIRCH 모델을 `n_clusters=None`으로 설정하여 학습시켜 보세요. 이 경우, BIRCH는 CF 트리의 리프 노드를 그대로 군집으로 간주합니다. 몇 개의 군집이 생성되는지 확인하고, `n_clusters=4`일 때의 결과와 어떻게 다른지 설명해보세요.

In [27]:
# 연습문제 7 코드
birch_none = Birch(n_clusters=None)
birch_labels_none = birch_none.fit_predict(X_processed)
n_clusters_birch_none = len(pd.unique(birch_labels_none))

print(f"n_clusters=None일 때 BIRCH가 찾은 군집 개수: {n_clusters_birch_none}")

n_clusters=None일 때 BIRCH가 찾은 군집 개수: 1996


---

### 7. OPTICS (Ordering Points To Identify the Clustering Structure)

<u>OPTICS</u>는 DBSCAN을 개선한 알고리즘으로, <u>밀도(density)</u>들을 처리하는 데 더 효과적입니다. DBSCAN이 고정된 `eps` 값 때문에 밀도가 다른 군집을 동시에 찾는 데 어려움을 겪는 문제를 해결하고자 제안되었습니다.

#### 🧠 개념 이해하기

OPTICS는 군집을 직접 할당하기보다는, 데이터 포인트의 순서를 재배치하여 군집 구조를 식별합니다. 각 포인트에 대해 <u>노이즈(Noise) 또는 이상치(Outlier)를 구분</u>라는 특별한 값을 계산합니다. 이는 해당 포인트가 얼마나 밀집된 지역에 있는지를 나타냅니다.

이 도달 가능성 거리를 순서대로 시각화한 것이 <u>핵심 파라미터:</u> 입니다. 
-   플롯에서 <u>포인트 유형:</u> 부분은 데이터가 밀집된 <u>핵심 포인트 (Core Point)</u>을 의미합니다.
-   <u>경계 포인트 (Border Point)</u> 부분은 밀도가 낮은 지역, 즉 군집 간의 경계나 <u>노이즈 (Noise Point)</u>를 의미합니다.

즉, OPTICS는 이 플롯을 통해 다양한 밀도 수준의 군집 구조를 한 번에 파악할 수 있게 해줍니다.

In [24]:
from sklearn.cluster import OPTICS

# OPTICS 모델 생성 및 학습
optics = OPTICS(min_samples=10)
optics_labels = optics.fit_predict(X_processed)

# 도달 가능성 거리와 순서 추출
reachability = optics.reachability_[optics.ordering_]
ordering = optics.ordering_

# 도달 가능성 플롯 시각화
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(len(reachability)), y=reachability, marker_color='blue'))
fig.update_layout(title='Reachability Plot (OPTICS)', 
                  xaxis_title='Data Point Order', 
                  yaxis_title='Reachability Distance')
fig.show()

In [25]:
# PCA를 이용한 군집 결과 시각화
df_pca['OPTICS_Cluster'] = optics_labels.astype(str)
fig_scatter = px.scatter(df_pca, x='PCA1', y='PCA2', color='OPTICS_Cluster',
                 title='OPTICS Clustering Result',
                 labels={'OPTICS_Cluster': '고객 군집'})
fig_scatter.show()

#### ✏️ 연습문제 8

OPTICS 모델의 `min_samples` 값을 20으로 늘려서 다시 학습하고, 도달 가능성 플롯과 군집화 결과를 `min_samples=10`일 때와 비교 분석하세요. 플롯의 골짜기 모양과 노이즈의 개수는 어떻게 변했나요?

In [28]:
# 연습문제 8 코드
optics_20 = OPTICS(min_samples=20)
optics_labels_20 = optics_20.fit_predict(X_processed)

# 도달 가능성 플롯 시각화
reachability_20 = optics_20.reachability_[optics_20.ordering_]
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(len(reachability_20)), y=reachability_20, marker_color='red'))
fig.update_layout(title='Reachability Plot (OPTICS, min_samples=20)', 
                  xaxis_title='Data Point Order', 
                  yaxis_title='Reachability Distance')
fig.show()

In [ ]:
# 연습문제 4 코드 (주관식 답변)